In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.dagcircuit import *
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.tools.visualization import dag_drawer
from qiskit.extensions.standard import CHGate, HGate, CnotGate, CyGate, CzGate
from qiskit.circuit import Measure
from qiskit.circuit.quantumregister import QuantumRegister, Qubit
from qiskit.circuit.classicalregister import ClassicalRegister, Clbit
from qiskit import BasicAer
from qiskit.visualization import plot_histogram
from qiskit import *
import numpy as np
import itertools
import networkx as nx
import pickle
from help_fun import cut_edge

q = QuantumRegister(4, 'q')
# c = ClassicalRegister(4, 'c')
circ = QuantumCircuit(q)
circ.h([q[2],q[1]])
circ.x([q[2],q[0]])
circ.y(q[3])
circ.cx(q[0], q[1])
circ.cy(q[2], q[3])
circ.ch(q[1], q[2])
circ.x(q[2])
# circ.barrier()
# circ.measure(q,c)
circ.draw()

original_dag = circuit_to_dag(circ)
print('original_dag has %d connected components' % nx.number_weakly_connected_components(original_dag._multi_graph))
# print(nx.algorithms.connectivity.cuts.minimum_edge_cut(dag._multi_graph))
dag_drawer(original_dag, filename='original_dag.pdf')
circ.draw(output='mpl',filename='original_circ.pdf')

cut_dag = cut_edge(original_dag=original_dag, wire=q[2],source_node_name='cy', dest_node_name='ch')
print('cut_dag has %d connected components' % nx.number_weakly_connected_components(cut_dag._multi_graph))
dag_drawer(cut_dag, filename='cut_dag.pdf')
dag_to_circuit(cut_dag).draw(output='mpl',filename='cut_circ.pdf')

original_dag has 1 connected components
cut_dag has 4 connected components


<Figure size 836.111x526.75 with 1 Axes>

In [2]:
for wire in cut_dag.wires:
    print('wire:', wire)
    [print('%s(%s)' % (x.type, x.name)) for x in cut_dag.nodes_on_wire(wire)]
    print()

wire: Qubit(QuantumRegister(4, 'q'), 0)
in(q[0])
op(x)
op(cx)
out(q[0])

wire: Qubit(QuantumRegister(4, 'q'), 1)
in(q[1])
op(h)
op(cx)
op(ch)
out(q[1])

wire: Qubit(QuantumRegister(4, 'q'), 2)
in(q[2])
op(h)
op(x)
op(cy)
op(measure)
out(q[2])

wire: Qubit(QuantumRegister(4, 'q'), 3)
in(q[3])
op(y)
op(cy)
out(q[3])

wire: Clbit(ClassicalRegister(1, 'cutC'), 0)
in(cutC[0])

wire: Qubit(QuantumRegister(1, 'cutQ'), 0)
in(cutQ[0])



In [3]:
for wire in original_dag.wires:
    print('wire:', wire)
    [print('%s(%s)' % (x.type, x.name)) for x in original_dag.nodes_on_wire(wire)]
    print()

wire: Qubit(QuantumRegister(4, 'q'), 0)
in(q[0])
op(x)
op(cx)
out(q[0])

wire: Qubit(QuantumRegister(4, 'q'), 1)
in(q[1])
op(h)
op(cx)
op(ch)
out(q[1])

wire: Qubit(QuantumRegister(4, 'q'), 2)
in(q[2])
op(h)
op(x)
op(cy)
op(ch)
op(x)
out(q[2])

wire: Qubit(QuantumRegister(4, 'q'), 3)
in(q[3])
op(y)
op(cy)
out(q[3])

